In [1]:
import numpy as np
import pandas as pd
import torch

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
val = pd.read_csv('data/val.csv')

In [3]:
X_train, y_train = train.drop('y',axis=1).to_numpy(), train['y'].to_numpy()
X_val, y_val = val.drop('y',axis=1).to_numpy(), val['y'].to_numpy()
X_test, y_test = test.drop('y',axis=1).to_numpy(), test['y'].to_numpy()


In [4]:
int_vocab = np.load('data/dict/int_vocab.npy', allow_pickle=True).item()
vocab_int = np.load('data/dict/vocab_int.npy', allow_pickle=True).item()

In [5]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
valid_data = TensorDataset(torch.from_numpy(X_val), torch.from_numpy(y_val))
test_data = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))

# dataloaders
batch_size = 32

# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [6]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print(f'Input size: {(sample_x.size())}') # batch_size, seq_length
print(f'Input: \n{sample_x}')
print()
print(f'Label size: {sample_y.size()}') # batch_size
print(f'Label: \n{sample_y}')

Input size: torch.Size([32, 2046])
Input: 
tensor([[    0,     0,     0,  ..., 22508, 39397,  4917],
        [    0,     0,     0,  ..., 19264, 55876, 49121],
        [    0,     0,     0,  ..., 38668, 60844, 12864],
        ...,
        [    0,     0,     0,  ..., 41344, 71285, 49478],
        [    0,     0,     0,  ..., 59883, 70072, 43854],
        [    0,     0,     0,  ..., 64357, 50407, 42483]])

Label size: torch.Size([32])
Label: 
tensor([0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 1, 0, 1])


In [7]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [8]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
        

In [9]:
# Instantiate the model with hyperparams
vocab_size = len(vocab_int)+1 # 
output_size = 1
embedding_dim = 512
hidden_dim = 256
n_layers = 2

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

SentimentRNN(
  (embedding): Embedding(74073, 512)
  (lstm): LSTM(512, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [10]:
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [11]:
# training params

epochs = 4 

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/4... Step: 100... Loss: 0.554723... Val Loss: 0.618392
Epoch: 1/4... Step: 200... Loss: 0.763447... Val Loss: 0.667735
Epoch: 1/4... Step: 300... Loss: 0.605592... Val Loss: 0.602839
Epoch: 1/4... Step: 400... Loss: 0.495310... Val Loss: 0.488665
Epoch: 2/4... Step: 500... Loss: 0.464275... Val Loss: 0.505512
Epoch: 2/4... Step: 600... Loss: 0.391303... Val Loss: 0.478726
Epoch: 2/4... Step: 700... Loss: 0.338647... Val Loss: 0.446847
Epoch: 2/4... Step: 800... Loss: 0.439647... Val Loss: 0.423173
Epoch: 3/4... Step: 900... Loss: 0.178129... Val Loss: 0.444712
Epoch: 3/4... Step: 1000... Loss: 0.375500... Val Loss: 0.426483
Epoch: 3/4... Step: 1100... Loss: 0.309075... Val Loss: 0.354917
Epoch: 3/4... Step: 1200... Loss: 0.159131... Val Loss: 0.326227
Epoch: 4/4... Step: 1300... Loss: 0.310586... Val Loss: 0.332713
Epoch: 4/4... Step: 1400... Loss: 0.148737... Val Loss: 0.397520
Epoch: 4/4... Step: 1500... Loss: 0.048105... Val Loss: 0.374620
Epoch: 4/4... Step: 1600... Loss: 

In [12]:
test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output, h = net(inputs, h)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.326
Test accuracy: 0.871


In [25]:
def pad_features(reviews_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
        :prams: reviews_ints: An array of tokenized words
        :prams: seq_length: length of resultant reviews
        :return: features: reviews_ints with seq_length feature
    '''
    
    # getting the correct rows x cols shape
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)

    # for each review, I grab that review and 
    for i, row in enumerate(reviews_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
    
    return features

In [38]:
from string import punctuation
def predict(net, test_review, sequence_length=200):
    
    net.eval()
    
    # tokenize review
    test_review = test_review.lower() # lowercase
    # get rid of punctuation
    test_text = ''.join([c for c in test_review if c not in punctuation])

    # splitting by spaces
    test_words = test_text.split()

    # tokens
    test_ints = []
    test_ints.append([vocab_int[word] for word in test_words])
    
    # pad tokenized sequence
    seq_length=sequence_length
    features = pad_features(test_ints, seq_length)
    
    # convert to tensor to pass into your model
    feature_tensor = torch.from_numpy(features)
    
    batch_size = feature_tensor.size(0)
    
    # initialize hidden state
    h = net.init_hidden(batch_size)
    
    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()
    
    # get the output from the model
    output, h = net(feature_tensor, h)
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze()) 
    # printing output value, before rounding
    print('Prediction value: {:.6f}'.format(output.item()))
    
    # print custom response
    if(pred.item()>=.5):
        print("Positive review detected!")
    else:
        print("Negative review detected.")

In [39]:
seq_length = 2046

review_positive_1 = 'This movie had the best acting and the dialogue was so good. I loved it.'
review_positive_2 = "I love this sofa. Its too comfortable...."

review_negative_1 = "This vaccum really sucks!!!"
review_negative_2 = "I received the food that is one year old. No one can even smell it."

In [40]:
print("TEST 1: \n Test for review_positive_1:")
predict(net, review_positive_1, seq_length)
print()
print("TEST 2: \n Test for review_positive_2:")
predict(net, review_positive_2, seq_length)
print()
print("TEST 3: \n Test for review_negative_1:")
predict(net, review_negative_1, seq_length)
print()
print("TEST 4: \n Test for review_negative_2:")
predict(net, review_negative_2, seq_length)
      

TEST 1: 
 Test for review_positive_1:
Prediction value: 0.910448
Positive review detected!

TEST 1: 
 Test for review_positive_2:
Prediction value: 0.899999
Positive review detected!

TEST 1: 
 Test for review_negative_1:
Prediction value: 0.401467
Negative review detected.

TEST 1: 
 Test for review_negative_2:
Prediction value: 0.086798
Negative review detected.
